In [1]:
from IPython.display import display, HTML
display(HTML("""<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{fontsize:12pt;padding:5px;}
table.dataframe{font-size:12px;}))
</style>
"""))

# <span style="color:red">ch09 07 LangChain과 vectorDatabase를 활용한 RAG구현 (UpstageEmbedding) </span>

[ RAG 구현 절차 ]

```
1. 문서의 내용을 읽는다 (document_loader를 이용)
    1) https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
    2) https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다 (한번에 이해하고 처리할 수 있는 입력 + 출력 토큰수가 제한)
    1) https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음
    1) OpenAIEmbeddings나 UpstageEmbeddings 이용해서 임베딩
    2) https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
    %pip install –q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음 (제공되는 Prompt활용)
    1) https://python.langchain.com/v0.2/docs/tutorials/rag/
    %pip install –q langchain langchainhub
    http://smith.langchain.com에서 key생성 .env key(LANGCHAIN_API_KEY) 추가
```

# 1. 문서를 쪼개면서 읽기 (o)

In [2]:
import time
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

start = time.time()
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(  # 문서를 쪼개는 기준이 문자수
    chunk_size = 1500,  #문서를 쪼갤때 1500글자씩 쪼개
    chunk_overlap = 200
)
# 1번째 chunk 1~1450글자
# 2번째 chunk 1250~1750글자
documents = loader.load_and_split(text_splitter = text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 시간 :', runtime)

문서 쪼개면서 읽는 시간 : 5.0091798305511475


# 2. 쪼갠 문서를 임베딩 → 벡터 데이터베이스 저장

- 임베딩 모델 : upstage의 text-embedding-3-large (기본 : text-embedding-ada-002)
- 벡터 데이터베이스 : chroma

In [3]:
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embeddings = UpstageEmbeddings(
    model = "solar-embedding-1-large"
    # model = "embedding-query"
)

In [4]:
doc_result = embeddings.embed_documents(
    ["소득세법 어쩌구 저쩌구", documents[0].page_content]
)
print(len(doc_result), len(doc_result[0]), len(doc_result[1]))

2 4096 4096


In [5]:
%%time

from langchain_chroma import Chroma

# 데이터를 처음 저장할 때
database = Chroma.from_documents(                                 
    documents=documents,
    embedding=embeddings,
    collection_name="tax-collection",  # 생략시 이름 랜덤
    persist_directory='./chroma_upstage'  # 생략시 로컬데이터베이스에 저장안됨. 프로그램 종료시 db날라감
)

# 이미 저장된 vector DB를 사용할 때
# database = Chroma(
#     embedding_function=embedding,
#     collection_name = "tax-collection",
#     persist_directory = './chroma'
# )

CPU times: total: 8.16 s
Wall time: 36.5 s


# 3. vector DB에 질문과 유사도 검색 (답변 생성을 위한 retrieval)

In [6]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query,
                                           k = 3)  # 기본 k는 4

In [7]:
retrieved_docs

[Document(id='fba1389a-6f11-483b-9ea8-5c56c7291282', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는

# 4. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4.1-nano")

In [9]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문자입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_docs}
Question : {query}"""

In [10]:
ai_message = llm.invoke(prompt)

In [11]:
print(ai_message.content)

연봉이 5천만원인 직장인의 소득세를 계산하기 위해서는 기본공제와 여러 공제 항목을 고려하여 과세표준을 산출하고, 그 과세표준에 따른 세율을 적용해야 합니다. 아래는 일반적인 계산 과정입니다.

1. 근로소득공제 계산
- 총급여액: 50,000,000원
- 근로소득공제 한도: 2,000만원
- 계산: 총급여액이 2천만원 이상이므로, 최대 공제액인 2,000만원 적용
- 근로소득공제액: 20,000,000원

2. 기타 공제 항목
- 기본공제 (본인): 150만원
- 자녀세액공제 등 기타 공제는 상세 정보가 없으니, 기본공제만 적용한다고 가정

3. 과세표준 계산
- 총급여액: 50,000,000원
- 공제합계 (근로소득공제 + 기본공제): 20,000,000원 + 1,500,000원 = 21,500,000원
- 과세표준: 50,000,000원 - 21,500,000원 = 28,500,000원

4. 소득세율 적용
- 과세표준 28,500,000원에 대한 세율은 24% 구간(1,200만원 초과 ~ 4,600만원 이하)이므로,
- 세액: 28,500,000원 × 24% = 6,840,000원

5. 세액공제 고려
- 근로소득세액공제는 일정 금액(예: 66만원~74만원)이 있으며, 계산 시 차감
- 예를 들어, 근로소득세액공제 66만원 적용
- 최종 세금: 6,840,000원 - 660,000원 = 6,180,000원

**요약:**  
이 직장인의 예상 소득세는 약 6,180,000원입니다.

※ 참고: 정확한 세액은 모든 공제 항목과 개별 세액공제, 세율 표에 따라 달라질 수 있으니, 국세청 홈택스의 근로소득 세액 계산기를 이용하거나 세무 전문가와 상담하시기를 권장합니다.


# 5. Augmentation을 위한 제공되는 Prompt활용하여 langchain으로 답변 생성

In [12]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)

```
 query → retriever 전달 (백터 검색 수행) 
 → retrieval문서 → prompt의 {context}에 삽입
 → query → prompt의 {question}에 삽입
```

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs = {'k':5}),
    chain_type_kwargs = {"prompt":prompt}
)

In [14]:
ai_message = qa_chain.invoke({"query":query})

In [15]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 근로소득공제(최대 2천만원) 후 과세표준에 세율을 적용하여 계산하며, 세율은 소득범위에 따라 다릅니다. 구체적인 금액은 제공된 정보만으로 계산하기 어렵지만, 예를 들어 근로소득공제 후 과세표준이 일정 수준일 경우 세율에 따라 대략 1백만원 내외의 세금이 부과될 수 있습니다. 정확한 세액 계산을 위해서는 총급여액, 근로소득공제, 기타 공제액 등을 상세히 알아야 합니다.'}